# Étape 1 : Prétraitement des Données


### 1. Chargement et Sélection des Données

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import glob # Pour trouver les fichiers
from google.colab import drive

drive.mount('/content/drive')
folder_path  = '/content/drive/MyDrive/Global Stock Market (2008-2023)/*.csv'
all_files = glob.glob(folder_path)

# Lire et combiner tous les fichiers CSV en un seul DataFrame
df_list = []
for filename in all_files:
    df_list.append(pd.read_csv(filename))

df = pd.concat(df_list, ignore_index=True)

print(df['Ticker'].unique())

ticker_valide = '^GSPC'
df_filtre = df[df['Ticker'] == ticker_valide].copy()
print(f"Nombre de lignes trouvées pour le ticker '{ticker_valide}': {len(df_filtre)}")

# Assurez-vous d'avoir des milliers de lignes maintenant !
colonnes_necessaires = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
df_aapl = df_filtre[colonnes_necessaires].copy()

df_aapl['Date'] = pd.to_datetime(df_aapl['Date'])
df_aapl.sort_values('Date', inplace=True) # Très important de trier les données par date !
df_aapl.set_index('Date', inplace=True)

close_prices = df_aapl['Close'].values.reshape(-1, 1)

ModuleNotFoundError: No module named 'google.colab'

### 2. Normalisation des Prix


In [2]:
# Normaliser les données entre 0 et 1
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_prices = scaler.fit_transform(close_prices)

### 3. Création de Séquences Temporelles

In [3]:
# Créer des séquences
sequence_length = 60
X, y = [], []

for i in range(len(scaled_prices) - sequence_length):
    X.append(scaled_prices[i:i + sequence_length])
    y.append(scaled_prices[i + sequence_length])

X, y = np.array(X), np.array(y)

### 4. Division des Données

In [4]:
# 80% pour l'entraînement, 10% pour la validation, 10% pour le test
train_size = int(len(X) * 0.8)
val_size = int(len(X) * 0.9)

X_train, X_val, X_test = X[:train_size], X[train_size:val_size], X[val_size:]
y_train, y_val, y_test = y[:train_size], y[train_size:val_size], y[val_size:]

### Étape 2 : Conception du Modèle LSTM


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Construire le modèle LSTM
model_lstm = Sequential([
    LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    Dropout(0.2),
    LSTM(units=50, return_sequences=False),
    Dropout(0.2),
    Dense(units=25),
    Dense(units=1)
])

# Compiler le modèle
model_lstm.compile(optimizer='adam', loss='mean_squared_error')

# Entraîner le modèle
history = model_lstm.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=32, epochs=50)

# Sauvegarder le modèle entraîné
model_lstm.save('lstm_stock_predictor.h5')

Epoch 1/50


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


93/93 ━━━━━━━━━━━━━━━━━━━━ 11s 84ms/step - loss: 0.0186 - val_loss: 8.9637e-04
Epoch 2/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 5s 53ms/step - loss: 0.0011 - val_loss: 0.0012
Epoch 3/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 7s 78ms/step - loss: 8.2503e-04 - val_loss: 0.0010
Epoch 4/50
44/93 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - loss: 7.6897e-04

### Générez toutes les prédictions LSTM en une seule fois :



In [ ]:
# Utilisez l'ensemble de données complet (X) pour générer les prédictions
all_lstm_preds_scaled = model_lstm.predict(X)

# Créez un DataFrame avec les prédictions
# Il y aura un décalage de 'sequence_length' jours
predictions_df = pd.DataFrame(all_lstm_preds_scaled, columns=['lstm_prediction'], index=df_aapl.index[sequence_length:])

# Fusionnez les prédictions avec votre DataFrame principal
df_aapl_with_preds = df_aapl.join(predictions_df)

# Supprimez les lignes sans prédiction
df_aapl_with_preds.dropna(inplace=True)

# Étape 3 & 4 : Développement et Intégration du DQN


### 1. Adapter l'Environnement de Trading

In [ ]:
from gymnasium.spaces import Box

import gymnasium as gym
from gymnasium import spaces
import pandas as pd
import numpy as np

from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler


class TradingEnv(gym.Env):
    """A simple stock trading environment"""
    def __init__(self, data: pd.DataFrame, initial_balance=10_000):
        super().__init__()
        self.data = data
        self.initial_balance = initial_balance
        self.current_step = 0

        # Action space: 0 = Hold, 1 = Buy, 2 = Sell
        self.action_space = spaces.Discrete(3)

        # Observation space: [balance, shares_held, net_worth, OHLCV data...]
        # +3 for balance, shares, and net worth
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(len(self.data.columns) + 3,), dtype=np.float32
        )

        # self.reset()

    def _get_observation(self):
        """Creates the observation vector for the current step."""
        obs = np.array([
            self.balance,
            self.shares_held,
            self.net_worth,
            *self.data.iloc[self.current_step].values
        ], dtype=np.float32)
        return obs

    def reset(self, seed=None):
        super().reset(seed=seed) # Important for gym compliance
        self.balance = self.initial_balance
        self.net_worth = self.initial_balance
        self.max_net_worth = self.initial_balance
        self.shares_held = 0
        self.current_step = 0 # Start from the beginning of the data
        self.done = False

        info = {} # Gym requires returning an info dictionary
        return self._get_observation(), info

    def step(self, action):
        """Execute one time step within the environment."""
        current_price = self.data['Close'].iloc[self.current_step]

        # Action: 1 = Buy
        if action == 1:
            # Buy one share
            if self.balance > current_price:
                self.shares_held += 1
                self.balance -= current_price
        # Action: 2 = Sell
        elif action == 2:
            # Sell one share
            if self.shares_held > 0:
                self.shares_held -= 1
                self.balance += current_price

        # Update net worth
        self.net_worth = self.balance + (self.shares_held * current_price)

        # Move to the next time step
        self.current_step += 1

        # Define the reward (e.g., change in net worth)
        reward = self.net_worth - self.initial_balance

        # Check if the episode is done
        if self.current_step >= len(self.data) - 1:
            self.done = True

        # Gym's step function returns 5 values
        observation = self._get_observation()
        terminated = self.done
        truncated = False # Typically used if a time limit is reached, not just the end of data
        info = {}

        return observation, reward, terminated, truncated, info

    def render(self, mode='human'):
        """Render the environment to the screen (optional)."""
        print(f'Step: {self.current_step}')
        print(f'Balance: {self.balance:.2f}')
        print(f'Shares held: {self.shares_held}')
        print(f'Net Worth: {self.net_worth:.2f}')



class TradingEnvWithLSTM(TradingEnv): # Hérite de votre classe TradingEnv
    def __init__(self, data: pd.DataFrame, lstm_model_path: str, sequence_length: int, initial_balance=10_000):
        # 1. Appeler le constructeur du parent en premier
        # Il va initialiser self.data, etc.
        super().__init__(data, initial_balance)

        # 2. Maintenant, initialiser tous les attributs spécifiques à l'enfant
        self.lstm_model = load_model(lstm_model_path)
        self.sequence_length = sequence_length

        # self.scaler et self.scaled_data dépendent de self.data, qui vient d'être créé par super()
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        self.scaled_data = self.scaler.fit_transform(self.data[['Close']].values)

        # 3. Mettre à jour l'espace d'observation qui a été défini par le parent
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(len(self.data.columns) + 3 + 1,), dtype=np.float32
        )


    def _get_lstm_prediction(self):
        """
        Génère une prédiction de prix à partir du modèle LSTM.
        """
        if self.current_step < self.sequence_length:
            return 0 # Pas assez de données pour prédire

        # Préparer la séquence pour le modèle LSTM
        start_index = self.current_step - self.sequence_length
        sequence = self.scaled_data[start_index:self.current_step]
        sequence = np.array([sequence]) # Reformater pour le modèle

        # Prédire et dé-normaliser la prédiction
        predicted_scaled = self.lstm_model.predict(sequence)[0][0]
        return predicted_scaled # On garde la valeur normalisée pour le DQN

    def _get_observation(self):
        """
        Crée le vecteur d'observation enrichi avec la prédiction LSTM.
        """
        lstm_pred = self._get_lstm_prediction()

        obs = np.array([
            self.balance,
            self.shares_held,
            self.net_worth,
            *self.data.iloc[self.current_step].values,
            lstm_pred # Ajouter la prédiction
        ], dtype=np.float32)

        return obs


    def reset(self, seed=None):
        # Call the parent's reset method to handle gym compliance
        super().reset(seed=seed)

        # Your custom logic for the child class
        self.balance = self.initial_balance
        self.shares_held = 0
        self.net_worth = self.initial_balance
        self.max_net_worth = self.initial_balance
        self.done = False

        # The initial step must be at least sequence_length to have a first prediction
        self.current_step = self.sequence_length

        # Return the observation AND an info dictionary
        return self._get_observation(), {}


### 2. Entraîner l'Agent DQN


In [ ]:
!pip install stable_baselines3

In [ ]:
from stable_baselines3 import DQN


# 1. Instancier l'environnement SIMPLE avec les données pré-calculées
# C'est la méthode la plus efficace.
env = TradingEnv(data=df_aapl_with_preds)

# 2. Créer et entraîner votre agent DQN
print("Création et entraînement du modèle DQN...")
model_dqn = DQN("MlpPolicy", env, verbose=1, tensorboard_log="./dqn_trading_tensorboard/")
model_dqn.learn(total_timesteps=100_000, progress_bar=True)

# 3. Sauvegarder le modèle final
print("Entraînement terminé. Sauvegarde du modèle...")
model_dqn.save("dqn_lstm_trader")

### Étape 5 : Évaluation


In [ ]:
# Préparer les données de test
test_data_df = df_aapl.iloc[val_size:]

# Créer un environnement de test
test_env = TradingEnvWithLSTM(
    data=test_data_df[['Open', 'High', 'Low', 'Close', 'Volume']],
    lstm_model_path='lstm_stock_predictor.h5',
    sequence_length=sequence_length
)

obs = test_env.reset()
done = False
while not done:
    action, _states = model_dqn.predict(obs, deterministic=True)
    obs, reward, done, info = test_env.step(action)
    test_env.render()

# Calculer le ROI (Return on Investment)
roi = ((test_env.net_worth - test_env.initial_balance) / test_env.initial_balance) * 100
print(f"Retour sur Investissement (ROI) final : {roi:.2f}%")